<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>Week 3 - Sequence Alignment</h1>

</div>

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

This week we'll look at some of the alignment algorithms discussed in lectures.

</div>

## Exercise 1: Hamming distance

Hamming distance is an ungapped measure of string distance. 

For sequences of same length, we iterate each position and note whether the letters match or mismatch. 

The returned distance score is the total number of mismatches. 



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<b>Challange:</b> Edit the Hamming distance function below so that it returns the correct Hamming distance for two strings `a` and `b`.
</div>

In [ ]:
def hamming(a, b):
    """
    Calculate the Hamming distance between strings a and b.
    The strings must be the same length.
    """
    # YOUR CODE HERE
    raise NotImplementedError

The cells below provide test input to check your code. 

In [ ]:
# Should return 1
hamming("GATTACA","GACTACA")

In [ ]:
# Should return 2
hamming("GATTACA","GACTACT")

In [ ]:
# What will your function do if the strings are of different length? 
# What should it do?
hamming("happiness", "applying")

When we add shifts to either sequence by adding or removing letters, hamming distance increases dramatically. 

This is problematic for biological sequences, as indels are a common form of genetic variation, and introduce these shifts. 

In the example below, inserting a base at the start of s2 (a single edit) results in a disproportionally high distance as all downstream bases are affected by the shift. 

We can rescue this effect by adding a corresponding gap to s1. 

In [ ]:
print(hamming("GATTACA","GATTACA"))    # Should return 0
print(hamming("GATTACA","TGATTAC"))    # Should return 6
print(hamming("-GATTAC","TGATTAC"))    # Should return 1

## Exercise 2: Levenshtein distance

Clearly, we need to be able to handle gaps. 

We need to move away from simple matches/mismatches and instead try to identify the smallest number of 'edits' which would transform one sequence into another. 

These edits include inserting / deleting characters. 

The idea of transformation & edits are ideal for sequence alignment as we're counting the number of evolutionary events which separate two sequences. 

Levenshtein distance is a formalisation of this idea, and can be implemented using two strategies: recursion, and dynamic programming.

Levenshtein distance builds upon hamming distance, by considering that a shift at the current location may result in better alignment of the remaining sequence. 


In [19]:
# at position 3, what shiting would return the best score?
# if keeping same alignment, only add an edit if characters don't match. 
# if adding a gap, need to add an edit (as an edit was made!).

# any sequence length difference will count towards to distance.
pos = 3
seq1 = "GATACA"
seq2 = "GATTACA"
#          ^

# Don't introduce a gap. 
# A CA 
# T ACA
# ^
this_dist = 1 if seq1[pos] != seq2[pos] else 0
future_dist = hamming('CA.', 'ACA')
print('inline: ', this_dist + future_dist)

# Introduce a gap in seq1. 
# - ACA 
# T ACA
# ^
this_dist = 1
future_dist = hamming('ACA', 'ACA')
print('seq1 gap: ', this_dist + future_dist)

# Introduce a gap in seq2
# A CA 
# - TACA
# ^
this_dist = 1 
future_dist = hamming('CA..', 'TACA')
print('seq2 gap: ', this_dist + future_dist)


inline:  4
seq1 gap:  1
seq2 gap:  4


This poses an issue. 

If we test 3 alignments at a given base, ***for each*** of these alignments, the next base also requires 3 alignments. 

This is inherently recursive. 


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<b>Challange:</b> Edit the `lev` function below to calculate Levenshtein distance recursively. You can use the costs 
* 1 for an indel
* 1 for a mismatch
* 0 for a match

This is the same function as shown during lectures, but try to implement it without looking back at the slides.

</div>

In [ ]:

def lev(a,b):
    """
    Recursively calculate Levenshtein distance between strings a and b.
    """
    # YOUR CODE HERE
    raise NotImplementedError

Try inserting a print statement at the top of your code to show the arguement values each time the function is called.

In [ ]:
print(lev("GATTACA","GAATACA")) # should return 1
print(lev("-GATTAC","TGATTAC")) # should return 1

In [ ]:
# Should return 4
lev("tuesday","sundays")

In [ ]:
# Should return 6
lev("happiness","applying")

## Exercise 3: Levenshtein distance with Dynamic Programming

During the recursive Levenshtein algorithm, each function call created 3 more function calls. <br>
This results in a huge number of operations, and is inefficient as we are repeating many calculations. 

Using an example from above, we can see why. <br>
Let's imagine that we don't consider shifts. Our recusive levenshtein becomes quite simple (essentially hamming distance).
- score += 1 if bases don't match 
- return score + lev(seq1[i+1], seq2[i+1])

Ie if we provide the strings 'GAATACA' and 'GATTACA' as input we would have the following calls: 
```python
0 + lev('AATACA', 'ATTACA')
0 + 0 + lev('ATACA', 'TTACA')
0 + 0 + 1 + lev('TACA', 'TACA')
0 + 0 + 1 + 0 + lev('ACA', 'ACA')
0 + 0 + 1 + 0 + 0 + lev('CA', 'CA')
0 + 0 + 1 + 0 + 0 + 0 + lev('A', 'A')
```

We can use dynamic programming to reduce the number of calculations we do. 

Dynamic programming approaches break one large problem into a series of local subproblems. <br>
In this setup, the solution to a given subproblem is only dependent on the previous solution. 

To adapt our levenshtein distance algorithm we will need to keep track of previous solutions. <br>
Additionally, we will need to calculate the current solution based on previous solutions, so will need to think about prefixes rather than suffixes. 

To implement this, we can create a matrix to keep track of already calculated levenshtein distances.  <br>
We will place levenshtein distances between all prefixes of string1 and all prefixes of string2 in this matrix.  <br>
To calculate the next distance, we therefore only need the relevant previous solutions, which are in the matix. 

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/tutorials/media/week3/levenshtein.png" width="600">

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Find the Levenshtein Distance of two squences using a dynamic programming approach. 

You can use the costs:
* 1 for an indel
* 1 for a mismatch
* 0 for a match

- [ ] Initialise the scoregrid as a numpy array
- [ ] Populate the first row and column with cumulative indel scores
- [ ] Fill the matrix (starting top left to right)
- [ ] Selecting the minimum scoring operation from {insertion, deletion, match, mismatch} at each step.

</div>

In [ ]:
import numpy as np

def levenshtein_distance(str1, str2):
    """
    Calculate the Levenshtein distance between two strings using dynamic programming.

    Parameters:
    str1 (str): The first input string.
    str2 (str): The second input string.

    Returns:
    int: The Levenshtein distance between the two input strings.
    """
    # YOUR CODE HERE 
    raise NotImplementedError

In [ ]:
# Test your function!
str1 = "kitten"
str2 = "sitting"
distance = levenshtein_distance(str1, str2)
print(f"Levenshtein distance between '{str1}' and '{str2}': {distance}")

<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>Extension Activities</h1>

</div>

From here, Levenshtein distance can be modified to create Needleman-Wunsch global alignment by simply changing to a penalty system. 

Local (Smith-Waterman) and semi-global alignment are also similar, but have a few extra tweaks. 

## Global alignment: Needleman-Wunsch

To do global alignment with the Needleman-Wunsch algorithm (dynamic programming), we need the following: 

1. Fill out the grid of alignment scores. This is enough to give the final ***alignment score***.
2. Fill out a separate grid which keeps track of the arrows (for each cell, did we come from diagonal, above, or left).
3. Perform traceback (using the arrows grid) to get the actual alignment of the strings.

Here we will just calculate the alignment score. We won't bother with items 2) and 3). 

Note: feel free to implement these yourself!

## Exercise 4: Needleman-Wunsch


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<b>Challange:</b> Complete the `calculate_scoregrid()` function to calculate the scores needed for global alignment via the Needleman-Wunsch algorithm.
</div>

In [ ]:
# A version with scores rather than costs, which can be specified
# Indels are scored per-base
def calculate_scoregrid(a, b, indel_score=-1, match_score=2, mismatch_score=-1):
    """
    Given two strings a and b, calculate the maximum score grid, using
    specified scores for indels, matches and mismatches. Return the grid.
    Grid row and column 0 correspond to "before" the start of each string,
    so grid indexes are offset by 1 from string indexes. That is,
    grid position [1,1] represents the result of matching a[0] to b[0].
    """
    # The grid needs to be 1 bigger in each direction than the string lengths
    X = len(a)+1
    Y = len(b)+1
    scoregrid = np.zeros((X,Y), int)
    
    # You need to:
    # * initialise the top edge of grid, i.e. scoregrid[x,0] for all x, with indel scores
    # * initialise the left edge of grid, i.e. scoregrid[0,y] for all x, with indel scores
    # * loop over x and y, filling out each cell of the grid by looking for the
    #   maximum possible score from each of the three earlier cells
    
    # YOUR CODE HERE 
    raise NotImplementedError

In [ ]:
# Once you've implemented calculate_scoregrid, this should show the correct
# values instead of all zeroes
a = "GATTACA"
b = "GACTATA"
scoregrid = calculate_scoregrid(a,b)
scoregrid

In [ ]:
print("Alignment score:",scoregrid[-1,-1])

## Exercise 5: Local Alignment


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<b>Challange:</b> Change your `calculate_scoregrid()` function to perform local instead of global alignment. You can import the `traceback_local()` function to help you test your result.
</div>

In [ ]:
# A version with scores rather than costs, which can be specified
# Indels are scored per-base
def calculate_scoregrid_local(a, b, indel_score=-1, match_score=2, mismatch_score=-1):
    """
    Given two strings a and b, calculate the maximum score grid, using
    specified scores for indels, matches and mismatches. Return the grid.
    Grid row and column 0 correspond to "before" the start of each string,
    so grid indexes are offset by 1 from string indexes. That is,
    grid position [1,1] represents the result of matching a[0] to b[0].
    """
    # The grid needs to be 1 bigger in each direction than the string lengths
    X = len(a)+1
    Y = len(b)+1
    scoregrid = np.zeros((X,Y), int)

    # YOUR CODE HERE 
    raise NotImplementedError

In [ ]:
# Test your function
a = 'happily'
b = 'applying'
scoregrid_local = calculate_scoregrid_local(a, b)
scoregrid_local